# LangChainの基本

## 1.準備

In [ ]:
# 必要なモジュールをインポート
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

# 環境変数の読み込み
load_dotenv()

# モデル名
MODEL_NAME = "gpt-5-mini"

## 2.言語モデルの呼出

In [ ]:
# モデルの作成
model = ChatOpenAI(model=MODEL_NAME)

# 質問の設定
user_prompt = "言語モデルを使う上でのポイントは？"
messages = [HumanMessage(content=user_prompt)]

# 言語モデルの呼出
response = model.invoke(messages)

# 結果を表示
print(response.content)

## 3.システムプロンプトとパラメータの設定

In [ ]:
# モデルの作成
model = ChatOpenAI(
    reasoning_effort="low",
    model=MODEL_NAME)

# 質問の設定
system_prompt = "あなたは小学校の先生です。例えを使って分かりやすく簡潔に答えます。漢字にはふりがなを振ります。"
user_prompt = "言語モデルを使う上でのポイントは？"
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt),
]

# 言語モデルの呼び出しと結果の表示（ストリーミング）
for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

## 4.プロンプトテンプレート

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "あなたは{input_language}から{output_language}に翻訳する優秀な翻訳家です。"
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template),
])

messages = chat_prompt.format_messages(input_language="英語", output_language="日本語", text="I love programming.")

# 作成されたプロンプト
messages


In [ ]:
# 作成したプロンプトを使用
# モデルの作成
model = ChatOpenAI(model=MODEL_NAME)

# 言語モデルの呼出
response = model.invoke(messages)

# 結果を表示
print(response.content)

## 5.Output parser

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

# モデルの作成
model = ChatOpenAI(model=MODEL_NAME)

# 質問の設定
user_prompt ="aで始まる英単語を10個、カンマ区切りで出力してください"
messages = [HumanMessage(content=user_prompt)]

# 言語モデルの呼出
response = model.invoke(messages)

# Output Parserの作成
output_parser = CommaSeparatedListOutputParser()

# Output parserで変換
word_list = output_parser.parse(response.content)
print(type(word_list))
print(word_list)

## 6.言語モデルによる構造化出力

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import List

# 1. 出力したいデータ構造（スキーマ）をPydanticで定義
class WordList(BaseModel):
    words: List[str] = Field(description="a list of words starting with 'a'")

# 2. モデルの作成
model = ChatOpenAI(model=MODEL_NAME) # Structured Output対応モデル推奨

# 3. モデルに構造化出力を強制する (ここでOpenAIのパラメータが設定されます)
structured_llm = model.with_structured_output(WordList)

# 4. 実行
user_prompt = "aで始まる英単語を10個出力してください"
response = structured_llm.invoke(user_prompt)

# 結果の確認
print(f"Type: {type(response.words)}")
print(f"List: {response.words}")

In [ ]:
from langchain_openai import ChatOpenAI

# 1. 出力してほしいデータ構造を「JSONスキーマ（辞書）」で定義
json_schema = {
    "title": "WordList",
    "description": "a list of words starting with 'a'",
    "type": "object",
    "properties": {
        "words": {
            "type": "array",
            "items": {"type": "string"},
            "description": "List of words"
        }
    },
    "required": ["words"]
}

# 2. モデルの作成
model = ChatOpenAI(model=MODEL_NAME)

# 3. method="json_mode" を指定して構造化モデルを作成
# これにより、戻り値が Pydanticオブジェクトではなく「dict」になります
structured_llm = model.with_structured_output(json_schema, method="json_mode")

# 4. 実行
# 【重要】json_modeを使う場合、プロンプトの中に必ず「JSON」という単語を含める必要があります
user_prompt = "aで始まる英単語を5つ、JSON形式で出力してください"
response = structured_llm.invoke(user_prompt)

# 結果の確認
print(f"Type: {type(response)}")
print(f"Dict: {response}")